In [12]:
from diffeq import *

## Basic vector operations

In [13]:
from diffeq.utils.vectors import *
A = vector(x = 10, y = 10)
B = vector(x = 3, y = -1, z = 1)

print('A:', A, sep = '\n'*2)
print('B:', B, sep = '\n'*2)
print('A + B:', A + B, sep = '\n'*2)
print('A - B:', A - B, sep = '\n'*2)
print('A @ B:', A @ B, sep = '\n'*2)
print('A * B:', A * B, sep = '\n'*2)
print('A / B:', A / B, sep = '\n'*2)
print('A ** B:', A ** B, sep = '\n'*2)
print('A**2:', A**2, sep = '\n'*2)
print('2**A:', 2**A, sep = '\n'*2)
print('A@A:', A@A)

A:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │10     │
└─────────────┘
B:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │3      │
│y    │-1     │
│z    │1      │
└─────────────┘
A + B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│x    │13      │
│z    │1.0     │
│y    │9       │
└──────────────┘
A - B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│x    │7        │
│z    │-1.0     │
│y    │11       │
└───────────────┘
A @ B:

20
A * B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│x    │30      │
│z    │0.0     │
│y    │-10     │
└──────────────┘
A / B:

┌─────────────────────────────┐
│axis │value                  │
├─────┼───────────────────────┤
│x    │3.3333333333333335     │
│z    │0.0                    │
│y    │-10.0                  │
└─────────────────────────────┘
A ** B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│x    │1000     │
│z    │0.0      │
│y    │0.1      │
└───────────────┘
A**2:

┌──────────

## Vector functions

In [14]:
@vector_function
def foo(x, y):
    return vector(
        x=10*y*x + y,
        y=x,
    )

v = vector(x = 10, y = 11)
print('value of input vector: ', v, sep = '\n')
print('function form: ', sep = '\n')
print(foo)
print('function output: ', sep = '\n')
print(foo(v))
print("function's yacobian: ", sep = '\n')
print(foo.yacobian)
print("function's yacobian value: ", sep = '\n')
print(foo.yacobian(v))

value of input vector: 
┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │11     │
└─────────────┘
function form: 
┌────────────────────────┐
│axis │function          │
├─────┼──────────────────┤
│x    │(y*x*10+y)        │
│y    │x                 │
└────────────────────────┘
function output: 
┌───────────────┐
│axis │value    │
├─────┼─────────┤
│x    │1111     │
│y    │10       │
└───────────────┘
function's yacobian: 
┌────────────────────────────┐
│axis     │function          │
├─────────┼──────────────────┤
│dx_dy    │(x*10+1.0)        │
│dx_dx    │(y*10)            │
│dy_dy    │0.0               │
│dy_dx    │1.0               │
└────────────────────────────┘
function's yacobian value: 
┌────────────────────┐
│axis     │value     │
├─────────┼──────────┤
│dx_dy    │101.0     │
│dx_dx    │110       │
│dy_dy    │0.0       │
│dy_dx    │1.0       │
└────────────────────┘


composition:

In [15]:
@vector_function
def foo1(x, y):
    return vector(
        x=10*y*x + y,
        y=-x + (-1),
    )

@vector_function
def foo2(x, y):
    return vector(
        x = 3*y - x,
        y = x - 1j
    )

In [16]:
@vector_function
def composed(x, y):
    return foo2(foo1(vector(x = x, y = y)))

A = vector(x = 1, y = 2)
print(composed)
print(composed(A))
print(foo2(foo1(A)))

print(composed.yacobian)

┌─────────────────────────────────────────┐
│axis │function                           │
├─────┼───────────────────────────────────┤
│x    │(((-x)+-1)*3+(-(y*x*10+y)))        │
│y    │(y*x*10+y+-1j)                     │
└─────────────────────────────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌───────────────────────────────────┐
│axis     │function                 │
├─────────┼─────────────────────────┤
│dx_dy    │((-(x*10+1.0))+3)        │
│dx_dx    │((-y*10)+-3.0)           │
│dy_dy    │(x*10+1.0)               │
│dy_dx    │(y*10)                   │
└───────────────────────────────────┘


In [17]:
print(composed.div)

┌───────────────────────────────────┐
│axis   │function                   │
├───────┼───────────────────────────┤
│div    │(x*10+(-y*10)+-2.0)        │
└───────────────────────────────────┘


In [18]:
print(composed.div.yacobian)

┌───────────────────────┐
│axis       │function   │
├───────────┼───────────┤
│ddiv_dy    │-10        │
│ddiv_dx    │10         │
└───────────────────────┘


## System of differential equations

In [19]:
solver = rk4_integrator(0.1)
sys = system(
    vector_function(lambda x, y:vector(x = -y, y = x)), solver, initials = vector(x = 1, y = 0)
)
results = sys.run(10)

In [20]:
print(results)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│axis    │value                                                                                                                                                                                              │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│x       │[1, 0.540302967116884, -0.41614526873411317, -0.9899919390660014, -0.6536459531899061, 0.28365810583410267, 0.9601684949770736, 0.7539057070501398, -0.14549338089595665, -0.9111266132598114]     │
│y       │[0, 0.8414704778002743, 0.9092979917935007, 0.1411224448434349, -0.7568001143085917, -0.9589251198182553, -0.27942016563257394, 0.6569818976735576, 0.989358664182

# Jacobian and Divergence 

In [21]:
# Define a vector field: F(x,y) = [x*y, x+y]
def field(state):
    x, y = state['x'], state['y']
    return vector({'x': x*y, 'y': x + y})

# Create vector function object
F = vector_function(lambda x, y: vector({'x': x*y, 'y': x + y}))

# Point to evaluate at
point = vector({'x': 2.0, 'y': 3.0})

# Compute Jacobian numerically
J_num = jacobian(field, point)
print("Numerical Jacobian:")
print(J_num)

# Compute divergence numerically
div_num = divergence(field, point)
print(f"\nNumerical Divergence: {div_num}")

Numerical Jacobian:
┌─────────────────────────────────┐
│axis     │value                  │
├─────────┼───────────────────────┤
│dx_dx    │3.0000000001232743     │
│dx_dy    │1.9999999999834963     │
│dy_dx    │0.9999999998437185     │
│dy_dy    │0.9999999998437185     │
└─────────────────────────────────┘

Numerical Divergence: 3.9999999999669926


## Visualization

In [22]:
from random import gauss
import diffeq.plotting.interactive as interactive
from diffeq import *

solver = rk4_integrator(0.001, 10)
a, b, c = -15, 35, -3/2
sys = system(
    vector_function(lambda x, y, z:vector(x = a*(x - y), y = b*x - y - z*x, z = x*y + c*z)), solver
)
trjs = []

for _ in range(10):
    sys.state = vector(x = gauss(), y = gauss(), z = gauss())
    results = sys.run(10)
    trjs.append(results)

interactive.generate_html(trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'lorenz', path = 'output/lorenz.html')

'\n<!DOCTYPE html>\n<html lang="ru">\n<head>\n    <meta charset="UTF-8">\n    <title>lorenz</title>\n    <style>\n        body { margin: 0; overflow: hidden; background-color: #050505; font-family: sans-serif; }\n        canvas { display: block; width: 100vw; height: 100vh; }\n        .info { position: absolute; top: 10px; left: 10px; color: #555; pointer-events: none; }\n    </style>\n</head>\n<body>\n<div class="info">lorenz</div>\n<canvas id="glCanvas"></canvas>\n\n<script id="vs" type="x-shader/x-vertex">#version 300 es\nlayout (location = 0) in vec3 aPos;\nlayout (location = 1) in vec4 aCol;\n\nout vec4 vColor;\n\nuniform float focus;\nuniform vec2 sphere_vector;\nuniform vec3 camera_position;\nuniform float screenK;\nuniform float time;\n\nmat2 rotate(float a){\n    return mat2(cos(a), sin(a), -sin(a), cos(a));\n}\n\nfloat get_t(vec3 F, vec3 P, vec3 camera_pos, float f){\n    return f / (dot(F, P - camera_pos) + 0.00001);\n}\n\nvoid main() {\n    vec3 D = vec3(0.0, 0.0, 1.0);\n  